In [10]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader 
import numpy as np
from scipy import io
import matplotlib.pyplot as plt
import argparse
import os
import copy
import time


def exact_soln(XYT):
    x, y, t = XYT[:,0], XYT[:,1], XYT[:,2]
    return 3*torch.sin(np.pi*x)*torch.sin(np.pi*y)*torch.exp(-2*(np.pi**2)*t) + torch.sin(3*np.pi*x)*torch.sin(np.pi*y)*torch.exp(-10*(np.pi**2)*t)


def stacked_grid(x,y,t):
    X, Y, T = torch.meshgrid(x, y, t)
    return torch.hstack((X.flatten()[:, None], Y.flatten()[:, None], T.flatten()[:,None])).float()


def cal_domain_grad(model, XYTGrid, device):
    Loss = torch.nn.MSELoss(reduction='mean')

    xyt = XYTGrid.requires_grad_(True).to(device)
    u = model.forward(xyt)
    u_grad = torch.autograd.grad(outputs=u, inputs=xyt, grad_outputs=torch.ones(u.shape).to(device), create_graph=True, allow_unused=True)[0]
    ux = u_grad[:,0]
    uy = u_grad[:,1]
    ut = u_grad[:,2]
    uxx = torch.autograd.grad(outputs=ux, inputs=xyt, create_graph=True, grad_outputs=torch.ones(ux.shape).to(device),allow_unused=True)[0][:,0]
    uyy = torch.autograd.grad(outputs=uy, inputs=xyt, create_graph=True, grad_outputs=torch.ones(uy.shape).to(device),allow_unused=True)[0][:,1]
    
    lossf = Loss(uxx+uyy-ut, torch.zeros_like(uxx, device=device).float())

    grad = torch.autograd.grad(outputs=lossf, 
                               inputs=xyt, 
                               grad_outputs=torch.ones(lossf.shape).to(device),
                               create_graph = True,
                               allow_unused=True)[0]
    
    return grad


class RADSampler():
    def __init__(self, Nf, device, k, c):    
        self.device = device
        self.k = k
        self.c = c
        self.Nf = Nf
        self.dense_Nf = Nf*1
        
    def update(self, model):
        
        x_new = torch.zeros(self.dense_Nf, 2, dtype = torch.float32, device=self.device).uniform_(0, 1)
        t_new = torch.zeros(self.dense_Nf, 1, dtype = torch.float32, device=self.device).uniform_(0, 0.1)
        x_t_new = torch.concatenate((x_new,t_new), axis = 1)
        
        XTGrid = torch.tensor(x_t_new, dtype = torch.float32, device=self.device, requires_grad=True) 
        XTGrid = XTGrid.to(self.device)

        xyt = XTGrid.requires_grad_(True).to(self.device)
        
        u = model.forward(xyt)
        u_grad = torch.autograd.grad(outputs=u, inputs=xyt, grad_outputs=torch.ones(u.shape).to(self.device), create_graph=True, allow_unused=True)[0]
        ux = u_grad[:,0]
        uy = u_grad[:,1]
        ut = u_grad[:,2]
        uxx = torch.autograd.grad(outputs=ux, inputs=xyt, create_graph=True, grad_outputs=torch.ones(ux.shape).to(self.device),allow_unused=True)[0][:,0]
        uyy = torch.autograd.grad(outputs=uy, inputs=xyt, create_graph=True, grad_outputs=torch.ones(uy.shape).to(self.device),allow_unused=True)[0][:,1]
        
        err = torch.abs((uxx+uyy-ut))
        err = (err**self.k)/((err**self.k).mean())+self.c
        err_norm = err/(err.sum())
        
        indice = torch.multinomial(err_norm, self.Nf, replacement = True)
        XTGrid = XTGrid[indice]
        self.XTGrid = XTGrid
        
class LASSampler():
    def __init__(self, Nf, fixed_uniform, device, L_iter = 1, beta = 0.2, tau = 0.002):
        self.Nf = Nf
        self.device = device
        self.cnt = 0
        self.beta = beta
        self.tau = tau
        self.L_iter = L_iter
        self.XTGrid = torch.tensor(copy.deepcopy(fixed_uniform), dtype=torch.float32, requires_grad=True).to(self.device)

    def update(self, phy_lf, model):

        x_new = torch.zeros(self.Nf, 2, dtype = torch.float32, device=self.device).uniform_(0, 1)
        t_new = torch.zeros(self.Nf, 1, dtype = torch.float32, device=self.device).uniform_(0, 0.1)
        x_t_new = torch.concatenate((x_new,t_new), axis = 1)
        self.XTGrid = x_t_new

        x_data = self.XTGrid
        samples = x_data.clone().detach().requires_grad_(True)
        
        for t in range(1, self.L_iter + 1):
            grad = phy_lf(model, samples, self.device)
            scaler = torch.sqrt(torch.sum((grad+1e-16)**2, axis = 1)).reshape(-1,1)
            grad = grad/scaler
            with torch.no_grad():
                samples = samples + self.tau * grad + self.beta*torch.sqrt(torch.tensor(2 * self.tau, device=self.device)) * torch.randn(samples.shape, device=self.device)
                samples[:, 0] = torch.clamp(samples[:, 0], min=0, max=1) 
                samples[:, 1] = torch.clamp(samples[:, 1], min=0, max=1)
                samples[:, 2] = torch.clamp(samples[:, 2], min=0, max=0.1)
            samples = samples.clone().detach().requires_grad_(True)
        self.XTGrid = samples.detach()

class L_INFSampler():
    def __init__(self, Nf, device, step_size = 0.05 , n_iter = 20):

        self.Nf = Nf
        self.device = device
        self.step_size = step_size
        self.n_iter = n_iter

    def update(self, phy_lf, model):

        x_new = torch.zeros(self.Nf, 2, dtype = torch.float32, device=self.device).uniform_(0, 1)
        t_new = torch.zeros(self.Nf, 1, dtype = torch.float32, device=self.device).uniform_(0, 0.1)
        x_t_new = torch.concatenate((x_new,t_new), axis = 1)
        self.XTGrid = x_t_new
            
        x_data = self.XTGrid
        samples = x_data.clone().detach().requires_grad_(True)
    
        for t in range(1, self.n_iter + 1):
            grad = phy_lf(model, samples, self.device)
            with torch.no_grad():
                samples = samples + self.step_size * torch.sign(grad)
                samples[:, 0] = torch.clamp(samples[:, 0], min=0, max=1)  
                samples[:, 1] = torch.clamp(samples[:, 1], min=0, max=1)
                samples[:, 2] = torch.clamp(samples[:, 2], min=0, max=0.1) 
            samples = samples.clone().detach().requires_grad_(True)
        self.XTGrid = samples.detach()        


class R3Sampler(nn.Module):
    def __init__(self,Nf, fixed_uniform, device):
        super(R3Sampler, self).__init__()
        self.Nf = Nf
        self.device = device
        self.XTGrid = torch.tensor(copy.deepcopy(fixed_uniform), dtype=torch.float32, requires_grad=True).to(self.device)
    
    def update(self, loss_aver, loss_ele):
        with torch.no_grad():
            cho_i = loss_ele > loss_aver
            cho_i = cho_i.to('cpu')
            self.XTGrid = self.XTGrid[cho_i].detach()
            need_n_sample = self.Nf-self.XTGrid.shape[0]
            x_new = torch.zeros(need_n_sample, 2, dtype = torch.float32, device=self.device).uniform_(0, 1)
            t_new = torch.zeros(need_n_sample, 1, dtype = torch.float32, device=self.device).uniform_(0, 0.1)
            x_t_new = torch.concatenate((x_new,t_new), axis = 1)
            self.XTGrid = torch.concatenate((self.XTGrid, x_t_new), axis = 0)
            self.XTGrid = torch.tensor(self.XTGrid, dtype = torch.float32, device=self.device, requires_grad=True)
    
class PINN(nn.Module):
    def __init__(self,k , c , t, exact_XYT, exact_u, LBs, UBs, Layers, N0, Nb, Nf, 
                 Activation = nn.Tanh(), 
                 model_name = "PINN.model", device = 'cpu',
                  display_freq = 100, samp = 'fixed' ):
        
        super(PINN, self).__init__()
        
        
        self.LBs = torch.tensor(LBs, dtype=torch.float32)
        self.UBs = torch.tensor(UBs, dtype=torch.float32)
        
        self.Layers = Layers
        self.in_dim  = Layers[0]
        self.out_dim = Layers[-1]
        self.Activation = Activation
        
        self.device = device
        
        x_init = torch.zeros(Nf, 2, dtype = torch.float32, device=self.device).uniform_(0, 1)
        t_init = torch.zeros(Nf, 1, dtype = torch.float32, device=self.device).uniform_(0, 0.1)
        x_t_init = torch.concatenate((x_init,t_init), axis = 1)
        self.fixed_uniform = torch.tensor(x_t_init, dtype = torch.float32, device=self.device, requires_grad=True)
        
        self.N0 = N0
        self.Nb = Nb
        self.Nf = Nf
        
        self.t = t
        self.exact_XYT = exact_XYT
        self.exact_u = exact_u.reshape(-1,1)
        
        self.XT0, self.u0  = self.InitialCondition(self.N0, self.LBs, self.UBs)
        self.left, self.right, self.top, self.bottom = self.BoundaryCondition(self.Nb, self.LBs, self.UBs)
        
        self.XT0 = self.XT0.to(device)
        self.u0 = self.u0.to(device) 
        
        self.left = self.left.to(device) 
        self.right = self.right.to(device)
        self.top = self.top.to(device) 
        self.bottom = self.bottom.to(device)
        
        self._nn = self.build_model()
        self._nn.to(self.device)
        self.Loss = torch.nn.MSELoss(reduction='mean')
        
        self.model_name = model_name
        self.display_freq = display_freq
        
        self.k = k
        self.c = c
        self.X_star = self.exact_XYT
        self.method = samp
        
        self.r3_sample = R3Sampler(self.Nf, self.fixed_uniform, device)
        self.las = LASSampler(self.Nf, fixed_uniform=self.fixed_uniform, device=self.device, L_iter = 1, beta = 0.2, tau=2e-3)
        self.l_inf = L_INFSampler(self.Nf, device=self.device, step_size = 0.05 , n_iter = 20)
        self.rad = RADSampler(Nf = self.Nf, device=self.device, k = self.k, c=self.c)
        
    
    def build_model(self):
        Seq = nn.Sequential()
        for ii in range(len(self.Layers)-1):
            this_module = nn.Linear(self.Layers[ii], self.Layers[ii+1])
            nn.init.xavier_normal_(this_module.weight)
            Seq.add_module("Linear" + str(ii), this_module)
            if not ii == len(self.Layers)-2:
                Seq.add_module("Activation" + str(ii), self.Activation)
        return Seq
        
    def forward(self, x):
        x = 2*(x - self.LBs.to(x.device)/((self.UBs - self.LBs).to(x.device))) - 1.0
        #print(3.5, x.shape)
        return self._nn.forward(x.to(self.device)).reshape(-1)

    def InitialCondition(self, N0, LB, UB):
        n_per_dim = int(np.round(np.sqrt(N0)))
        t_in = torch.tensor([LB[-1]]).float()
        x_in = torch.linspace(LB[0], UB[0], n_per_dim).float()
        y_in = torch.linspace(LB[1], UB[1], n_per_dim).float()
        XYT_in = stacked_grid(x_in, y_in, t_in)
        u0 = exact_soln(XYT_in)
        return XYT_in, u0

    def BoundaryPoints(self,nb, xb, yb, LBs, UBs, where = 'left'):
        n_per_dim = int(np.round(np.sqrt(nb)))
        if where in ['left', 'right']:
            Xb = torch.tensor([xb]).float()
            Yb = torch.linspace(LBs[1], UBs[1], n_per_dim).float()
        else:
            Yb = torch.tensor([yb]).float()
            Xb = torch.linspace(LBs[0], UBs[0], n_per_dim).float()
        Tb = torch.linspace(LBs[-1], UBs[-1], nb).float()
        return stacked_grid(Xb,Yb,Tb)

    
    def BoundaryCondition(self,Nb, LBs, UBs):
        ## left boundary: 
        ## Choose Nb time instances on x = LB, x = UB, y = LB, and y = UB
        nb = int(np.round(Nb/4))
        XYTleft = self.BoundaryPoints(nb, LBs[0], LBs[1], LBs, UBs, 'left')
        XYTright = self.BoundaryPoints(nb, UBs[0], LBs[1], LBs, UBs, 'right')
        XYTtop = self.BoundaryPoints(nb, LBs[0], UBs[1], LBs, UBs, 'top')
        XYTbottom = self.BoundaryPoints(nb, LBs[0], LBs[1], LBs, UBs, 'bottom')
        return XYTleft, XYTright, XYTtop, XYTbottom
    
    def ICLoss(self):
        uv0_pred = self.forward(self.XT0)
        loss = self.Loss(uv0_pred, self.u0.to(self.device))
        return loss
        
    def BCLoss(self):
        U_L, U_R, U_T, U_B = self.forward(self.left), self.forward(self.right), self.forward(self.top), self.forward(self.bottom)
        ULx, URx, UTx, UBx = exact_soln(self.left).to(self.device), exact_soln(self.right).to(self.device), \
                             exact_soln(self.top).to(self.device), exact_soln(self.bottom).to(self.device)
        return self.Loss(U_L, ULx) + self.Loss(U_R, URx) + \
               self.Loss(U_T, UTx) + self.Loss(U_B, UBx)
    
    def PhysicsLoss(self, XTGrid):
        xyt = XTGrid.requires_grad_(True).to(self.device)
        u = self.forward(xyt)
        u_grad = torch.autograd.grad(outputs=u, inputs=xyt, grad_outputs=torch.ones(u.shape).to(self.device), create_graph=True, allow_unused=True)[0]
        ux = u_grad[:,0]
        uy = u_grad[:,1]
        ut = u_grad[:,2]
        uxx = torch.autograd.grad(outputs=ux, inputs=xyt, create_graph=True, grad_outputs=torch.ones(ux.shape).to(self.device),allow_unused=True)[0][:,0]
        uyy = torch.autograd.grad(outputs=uy, inputs=xyt, create_graph=True, grad_outputs=torch.ones(uy.shape).to(self.device),allow_unused=True)[0][:,1]

        loss2 = (uxx+uyy-ut)**2
        loss1 = loss2.mean()
             
        return loss1, loss2 

    def Train(self, n_iters, weights=(1.0,1.0,1.0)):
        params = list(self.parameters())
        optimizer = optim.Adam(params, lr=1e-3)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5000, gamma=0.9, last_epoch=-1)
        min_loss = 999999.0
        Training_Losses = [-10]*n_iters
        Test_Losses = []
        rel_error = [-10]*(1+n_iters//1000)
        
        for jj in range(n_iters):
            Total_ICLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
            Total_BCLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
            Total_PhysicsLoss = torch.tensor(0.0, dtype = torch.float32, device=self.device, requires_grad = True)
            
            Total_ICLoss = Total_ICLoss + self.ICLoss()
            Total_BCLoss = Total_BCLoss + self.BCLoss()
            
            if self.method =='r3':
                if jj == 0:
                    XTGrid = self.r3_sample.XTGrid
                    XTGrid = torch.tensor(XTGrid, dtype = torch.float32, device=self.device, requires_grad=True) 
                else:
                    with torch.no_grad():
                        self.r3_sample.update(loss1, loss2)
                        XTGrid = self.r3_sample.XTGrid
                        XTGrid = torch.tensor(XTGrid, dtype = torch.float32, device=self.device, requires_grad=True) 
                        
            elif self.method == 'las':
                if jj == 0:
                    XTGrid = self.las.XTGrid
                    XTGrid = torch.tensor(XTGrid, dtype=torch.float32, requires_grad=True).to(self.device)
                else:
                    if self.las.cnt % 1 == 0:# 4,6,8,10 cnt = 4, 
                        self.las.update(cal_domain_grad, self._nn)
                    XTGrid = self.las.XTGrid
                    XTGrid = torch.tensor(XTGrid, dtype=torch.float32, requires_grad=True).to(self.device)
                self.las.cnt += 1
            
            elif self.method =='l_inf':
                    self.l_inf.update(cal_domain_grad, self._nn)
                    XTGrid = self.l_inf.XTGrid
                    XTGrid = torch.tensor(XTGrid, dtype=torch.float32, requires_grad=True).to(self.device)
                
            elif self.method =='rad':
                    self.rad.update(self._nn)
                    XTGrid = self.rad.XTGrid
                    XTGrid = torch.tensor(XTGrid, dtype=torch.float32, requires_grad=True).to(self.device)   
            
            elif self.method =='fixed':
                    XTGrid = torch.tensor(self.fixed_uniform, dtype = torch.float32, device=self.device, requires_grad=True) 
            
            elif self.method =='random-r':
                    x_new = torch.zeros(self.Nf, 2, dtype = torch.float32, device=self.device).uniform_(0, 1)
                    t_new = torch.zeros(self.Nf, 1, dtype = torch.float32, device=self.device).uniform_(0, 0.1)
                    x_t_new = torch.concatenate((x_new,t_new), axis = 1)
                    XTGrid = torch.tensor(x_t_new, dtype = torch.float32, device=self.device, requires_grad=True) 
                
            optimizer.zero_grad()    
            loss1, loss2 = self.PhysicsLoss(XTGrid) # For r3 method, loss2 contains element-wise errors
            
            Total_PhysicsLoss = Total_PhysicsLoss + loss1
            Total_Loss = weights[0]*Total_ICLoss + weights[1]*Total_BCLoss\
                        + weights[2]*Total_PhysicsLoss 
            
            Total_Loss.backward()
            optimizer.step()
            scheduler.step()
            if Total_Loss < min_loss:
                torch.save(self._nn.state_dict(), "../models/"+self.method+'_'+str(len(self.Layers)-2)+'_'+str(self.Nf)+'.pt')
                min_loss = float(Total_Loss)
                    
            Training_Losses[jj] = float(Total_Loss)
            
            if (jj+1) % self.display_freq == 0:
                with torch.no_grad():
                    outputs = self.forward(self.X_star)
                    outputs = outputs.reshape(-1,1)
                    re = np.linalg.norm(self.exact_u.cpu()-outputs.cpu().detach()) / np.linalg.norm(self.exact_u.cpu().detach())
                    rel_error[int((jj+1)/1000)] = float(re*100)
                    
                print("Iteration Number = {}".format(jj+1))
                print("\tIC Loss = {}".format(float(Total_ICLoss)))
                print("\tBC Loss = {}".format(float(Total_BCLoss)))
                print("\tPhysics Loss = {}".format(float(Total_PhysicsLoss)))
                print("\tTraining Loss = {}".format(float(Total_Loss)))
                print("\tRelative L2 error (test) = {}".format(float(re*100)))

        return Training_Losses, rel_error


if __name__ == "__main__":
                
        parser = argparse.ArgumentParser()
        parser.add_argument('--nodes', type=int, default = 128, help='The number of nodes per hidden layer in the neural network')
        parser.add_argument('--layers', type=int, default = 8, help='The number of hidden layers in the neural network')
        parser.add_argument('--N0', type=int, default = 100, help='The number of points to use on the initial condition')
        parser.add_argument('--Nb', type=int, default = 100, help='The number of points to use on the boundary condition')
        parser.add_argument('--Nf', type=int, default = 1000, help='The number of collocation points to use')
        parser.add_argument('--epochs', type=int, default = 200000, help='The number of epochs to train the neural network')
        parser.add_argument('--method', type=str, default='random-r', help='Sampling method') # fixed, random-r, rad, r3, l_inf, las 
        parser.add_argument('--model-name', type=str, default='PINN_model', help='File name to save the model')
        parser.add_argument('--display-freq', type=int, default=1000, help='How often to display loss information')
        parser.add_argument('-f')
        args = parser.parse_args()


        if not os.path.exists("../models/"):
            os.mkdir("../models/")
            
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        NHiddenLayers = args.layers
        
        LBs = [0, 0, 0.0]
        UBs = [1.0, 1.0, 0.1]

        Nx, Ny, Nt = 100, 100, 20
        X = torch.linspace(LBs[0], UBs[0], Nx).float()
        Y = torch.linspace(LBs[1], UBs[1], Ny).float()
        T = torch.linspace(LBs[2], UBs[2], Nt).float()
        XYT = stacked_grid(X,Y,T)
        Exact_U = exact_soln(XYT)

        Layers = [3] + [args.nodes]*NHiddenLayers + [1]
        Activation = nn.Tanh()

        k = 1
        c = 1

        repeat = [0, 1, 2, 3, 4]
        for i in repeat:
            pinn = PINN(  k = k,
                          c = c,
                          t= T,
                          exact_XYT = XYT,
                          exact_u = Exact_U,
                          Layers = Layers,
                          LBs = LBs,
                          UBs = UBs,
                          N0 = args.N0,
                          Nb = args.Nb,
                          Nf = args.Nf,
                          Activation = Activation,
                          device = device,
                          model_name = "../models/" + args.model_name + ".model_"+args.method+'_'+str(args.layers)+'_'+str(args.Nf)+'_'+str(i),
                          display_freq = args.display_freq, samp = args.method )

            Losses_train, Losses_rel_l2 = pinn.Train(args.epochs, weights = (1, 1, 1)) # initial, boundary, residual

            torch.save(Losses_train, "../models/" + args.model_name + ".loss_"+args.method+'_'+str(args.layers)+'_'+str(args.Nf)+'_'+str(i))
            torch.save(Losses_rel_l2, "../models/" + args.model_name + ".rel_l2_"+args.method+'_'+str(args.layers)+'_'+str(args.Nf)+'_'+str(i))

/tmp/ipykernel_1056366/301632227.py:184: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.fixed_uniform = torch.tensor(x_t_init, dtype = torch.float32, device=self.device, requires_grad=True)
/tmp/ipykernel_1056366/301632227.py:148: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.XTGrid = torch.tensor(copy.deepcopy(fixed_uniform), dtype=torch.float32, requires_grad=True).to(self.device)
/tmp/ipykernel_1056366/301632227.py:90: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.XTGrid = torch.tensor(copy.deepcopy(fixed_uniform), dtype=

Iteration Number = 1000
	IC Loss = 1.5826504230499268
	BC Loss = 0.1788272261619568
	Physics Loss = 0.005858185235410929
	Training Loss = 1.7673358917236328
	Relative L2 error (test) = 81.64623975753784


KeyboardInterrupt: 